<a href="https://colab.research.google.com/github/atakeuchi/utils/blob/main/Use_Neptune_API_to_log_your_first_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Neptune API to Iog your first experiment

## Before you start

In [ ]:
#install packages
!pip install --quiet lightgbm neptune-client neptune-contrib scikit-plot matplotlib==3.2.0 plotly==4.12.0


## Step 1 - Initialize Neptune

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# connect to the account
import neptune
with open('/content/drive/MyDrive/neptune_tk.txt', 'r') as file:
    api_token = file.read().replace('\n', '')

neptune.init(project_qualified_name='atakeuchi/sandbox', # change this to your `workspace_name/project_name`
             api_token=api_token
            )

In [ ]:
from neptunecontrib.monitoring.lightgbm import neptune_monitor

params = {'boosting_type': 'gbdt',
      'objective': 'multiclass',
      'num_class': 3,
      'num_leaves': 31,
      'learning_rate': 0.05,
      'feature_fraction': 0.9
      }
# parameter log
neptune.create_experiment(name='LightGBM-training', params=params)


gbm = lgb.train(params,
        lgb_train,
        num_boost_round = 500,
        valid_sets = [lgb_train, lgb_eval],
        valid_names = ['train','valid'],
        callbacks = [neptune_monitor()], # Just add this callback
       )

# model artifacts
gbm.save_model('lightgbm.pkl')

# Log model
neptune.log_artifact('lightgbm.pkl')

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_roc
from neptunecontrib.api import log_chart

y_test_pred = model.predict(x_test)
fig, ax = plt.subplots()
plot_roc(y_test, y_test_pred, ax=ax)

# save charts
log_chart(name='ROC curve', chart=fig)

In [ ]:
neptune.stop()